#### Import Python packages 

In [23]:
# Import Python packages 
import cassandra
import re

# Part II. Complete the Apache Cassandra coding portion of your project. 

## After processing a single CSV file titled <font color=red>event_datafile_new.csv</font>, is generated within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Create Apache Cassandra infrastructure

#### Creating a Cluster

In [24]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [25]:
# Create a Keyspace 
try :
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION = {
                    'class': 'SimpleStrategy', 'replication_factor': 1 } """)
except Exception as e:
    print("Error: Issue in creating KeySapce ")
    print(e)

#### Set Keyspace

In [26]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print("Error: Issue in setting Keyspace")
    print(e)


## REMEMBER: With Apache Cassandra you model the database tables on the queries you want to run.
Each time you run SELECT queries, remember to execute scripts in the sequence.  `create_tables.py` and `etl.py`files so that event log data is extracted, processed and loaded into the database tables.


## Create queries to ask the following three questions of the data , a sample queries 

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Run SELECT to verify that the data have been inserted into each table

In [27]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist_name, song_title, duration FROM music_library \
         WHERE session_id = 139 AND item_in_session = 4"

try :
    rows = session.execute(query)
    
except Exception as e :
    print(e)
    
for row in rows :
    print (row.artist_name, row.song_title, row.duration)

Tamba Trio Quem Quiser Encontrar O Amor 177.18812561035156


In [28]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

select_query = "SELECT artist_name, song_title, item_in_session, first_name, last_name, session_id \
                FROM user_music_library \
                WHERE user_id = 8 AND session_id = 139 \
                ORDER BY item_in_session DESC"

try :
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.first_name, row.last_name, row.song_title, row.session_id, row.item_in_session)


Girl Talk Kaylee Summers Once again 139 8
Blue October / Imogen Heap Kaylee Summers Congratulations 139 7
Infected Mushroom Kaylee Summers Becoming Insane 139 6
The Mars Volta Kaylee Summers Eriatarka 139 5
Tamba Trio Kaylee Summers Quem Quiser Encontrar O Amor 139 4
Mr Oizo Kaylee Summers Flat 55 139 3
Des'ree Kaylee Summers You Gotta Be 139 1


In [29]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Select query to validate the data is entered into database table        
select_query = "SELECT song_title, first_name, last_name, artist_name, user_id \
                FROM user_music_history \
                WHERE song_title = 'Once again'"

try :
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    

for row in rows:
    print (row.song_title, row.first_name, row.last_name, row.artist_name, row.user_id)
    

Once again Kaylee Summers Girl Talk 8


### Drop the tables before closing out the sessions

In [30]:
## Drop the tables before closing out the sessions

drop_query1 = "DROP TABLE IF EXISTS udacity.music_library"
try :
    session.execute(drop_query1)
except Exception as e:
    print(e)
    
drop_query2 = "DROP TABLE IF EXISTS udacity.user_music_library"
try :
    session.execute(drop_query2)
except Exception as e:
    print(e)
    
drop_query3 = "DROP TABLE IF EXISTS udacity.user_music_history"
try :
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()